# Sugges_

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest. This notebook contains implementation of [Trie](https://en.wikipedia.org/wiki/Trie) datastructure (prefix tree), that uses in order to complete user query.

## Install Trie DS support

https://github.com/google/pygtrie

In [1]:
!pip install pygtrie

In [2]:
import pygtrie
t = pygtrie.CharTrie()
t["this is 1"] = "A"
t["this is 2"] = "B"
t["that is 3"] = "C"

print(t)

n = t.has_node('this') == pygtrie.Trie.HAS_VALUE
s = t.has_node('this') == pygtrie.Trie.HAS_SUBTRIE

print(f"Node = {n}; Subtree = {s}")

for key, val in t.iteritems("this"):
    print(key, '~', val)

CharTrie(this is 1: A, this is 2: B, that is 3: C)
Node = False; Subtree = True
this is 1 ~ A
this is 2 ~ B


## 1. Build a trie upon a dataset ##

### 1.1 Read dataset

Download the [dataset](https://drive.google.com/drive/folders/1rOE5eed37Jy2ANQItZVwDIFgPmkCoFu6)

In [3]:
#TODO: Read the dataset
import pandas as pd
import sys

# value of the worst rank, which means that user have not clicked any url by the query
NON_CLICKED_FLAG = sys.maxsize

aol_data = pd.read_csv("user-ct-test-collection-01.txt", sep="\t")

# replace NaN with sys.maxsize for ItemRank and with '' for ClickURL columns 
# just for more convenient processing.
aol_data['ClickURL'] = aol_data['ClickURL'].replace(pd.np.nan, '', regex=True)
aol_data['ItemRank'] = aol_data['ItemRank'].replace(pd.np.nan, NON_CLICKED_FLAG, regex=True)

print("DS size:", aol_data.shape[0])
print("DS head:")
print(aol_data.head())
print("DS tail:")
print(aol_data.tail())

DS size: 3558411
DS head:
   AnonID                        Query            QueryTime      ItemRank  \
0     142               rentdirect.com  2006-03-01 07:17:12  9.223372e+18   
1     142  www.prescriptionfortime.com  2006-03-12 12:31:06  9.223372e+18   
2     142                   staple.com  2006-03-17 21:19:29  9.223372e+18   
3     142                   staple.com  2006-03-17 21:19:45  9.223372e+18   
4     142    www.newyorklawyersite.com  2006-03-18 08:02:58  9.223372e+18   

  ClickURL  
0           
1           
2           
3           
4           
DS tail:
           AnonID                      Query            QueryTime  \
3558406  24968114                          -  2006-05-31 01:04:20   
3558407  24969251  sp.trafficmarketplace.com  2006-05-31 15:51:23   
3558408  24969374            orioles tickets  2006-05-31 12:24:51   
3558409  24969374            orioles tickets  2006-05-31 12:31:57   
3558410  24969374          baltimore marinas  2006-05-31 12:43:40   

         

### 1.2 Build Trie

Suggest function have to be non-sensitive to stop words because user can confuse/omit prepositions.

Trie is based on the dataset, query statistics such as query frequency, urls and ranks in nodes are stored. Some queries may not have associated urls, others may have multiple ranked urls (in this case only the most relevant one is stored for reasons of memory saving).

In [4]:
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

class Preprocessor:
    """Class for preprocessing of the extracted queries."""
    
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))

    def tokenize(self, text):
        return nltk.word_tokenize(text)
    
    def preprocess(self, text):
        """
        Tokenize lowercased text, and remove stop-words. 
        Returns queries without stop-words.
        """
        text = str(text).lower().strip()
        tokenized_text = self.tokenize(text)
        return " ".join([word for word in tokenized_text if word not in self.stop_words])

[nltk_data] Downloading package stopwords to /home/misha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
preprocessor = Preprocessor()

# create a column in the dataframe, which contains query without stop-words.
aol_data['processed_query'] = [preprocessor.preprocess(query) for query in aol_data['Query']]

print(aol_data.tail())

           AnonID                      Query            QueryTime  \
3558406  24968114                          -  2006-05-31 01:04:20   
3558407  24969251  sp.trafficmarketplace.com  2006-05-31 15:51:23   
3558408  24969374            orioles tickets  2006-05-31 12:24:51   
3558409  24969374            orioles tickets  2006-05-31 12:31:57   
3558410  24969374          baltimore marinas  2006-05-31 12:43:40   

             ItemRank                   ClickURL            processed_query  
3558406  9.223372e+18                                                     -  
3558407  9.223372e+18                             sp.trafficmarketplace.com  
3558408  9.223372e+18                                       orioles tickets  
3558409  2.000000e+00  http://www.greatseats.com            orioles tickets  
3558410  9.223372e+18                                     baltimore marinas  


In [6]:
QUERY_COUNT_KEY = 'count'
RANK_KEY = 'rank'
URL_KEY = 'url'


def build_aol_trie():
    """ build trie based on data """ 
    
    def find_query_in_aol_trie(similar_queries, query, query_rank, query_url):
        if query in similar_queries:
            query_params = similar_queries[query]
            query_params[QUERY_COUNT_KEY] += 1  # increase occurence of query met
            
            # keep just url with best rank for saving memory:
            if query_rank < query_params[RANK_KEY]:
                query_params[RANK_KEY] = query_rank
                query_params[URL_KEY] = query_url
        else:
            similar_queries[query] = {QUERY_COUNT_KEY: 1, 
                                      RANK_KEY: query_rank, 
                                      URL_KEY: query_url}
    
    aol_trie = pygtrie.CharTrie()
    for processed_query, query, rank, url in zip(aol_data['processed_query'],
                                                 aol_data['Query'],
                                                 aol_data['ItemRank'],
                                                 aol_data['ClickURL']):
        if processed_query not in aol_trie:
            aol_trie[processed_query] = dict()

        find_query_in_aol_trie(aol_trie[processed_query], query, rank, url)
    
    return aol_trie

aol_trie = build_aol_trie()

In [7]:
# test trie
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)

sample question surveys ~ {'sample question surveys': {'count': 5, 'rank': 1.0, 'url': 'http://www.lg-employers.gov.uk'}}
sample questions immigration interview ~ {'sample questions for immigration interview': {'count': 1, 'rank': 9.223372036854776e+18, 'url': ''}}
sample questions interview ~ {'sample questions for interview': {'count': 1, 'rank': 1.0, 'url': 'http://www.quintcareers.com'}}
sample questions family interview ~ {'sample questions for family interview': {'count': 3, 'rank': 2.0, 'url': 'http://www.grandparents-day.com'}}
sample questions sociology race ethnicity ~ {'sample questions sociology race and ethnicity': {'count': 1, 'rank': 9.223372036854776e+18, 'url': ''}}
sample questions biology ~ {'sample questions biology': {'count': 2, 'rank': 3.0, 'url': 'http://www.utexas.edu'}}
sample questions us citizenship test ~ {'sample questions for us citizenship test': {'count': 1, 'rank': 1.0, 'url': 'http://uscis.gov'}}
sample questionarie teaching evaluation ~ {'sample ques

## 2. Implementation of a suggest function which is non-sensitive to stop words ##

Suggest options for user query based on Trie.
Output results sorted by frequency, query count for each suggestion is printed. If there is an url available, it is also printed. If multiple url-s are available, the one with the highest rank (the less the better) is printed.

In [8]:
def complete_user_query(query, trie, top_k=5):
    """ suggest top_k options for a user query.
    Sort results by frequency, suggest first ranked urls if available""" 
    
    def find_complete_queries(preprocessed_query, completed_queries):
        """make search in trie by given preprocessed_query, and return sorted (by frequency) results."""
        for full_processed_queries, full_init_queries_dict in trie.iteritems(preprocessed_query):
            for full_init_query, init_query_param in full_init_queries_dict.items():
                completed_queries.append((init_query_param[QUERY_COUNT_KEY], full_init_query, init_query_param[URL_KEY]))

        return sorted(completed_queries, key=lambda item: item[0], reverse=True)[:top_k]
    
    
    if len(query) == 0:  # just check on dull case:
        print("Your query is empty")
        return
        
    completed_queries = []
    preprocessed_query = preprocessor.preprocess(query)
    
    if len(preprocessed_query) == 0:  # can be true if query consist only of stopwords.
        preprocessed_query = query  # try to find at least smth.

    if trie.has_subtrie(preprocessed_query) or trie.has_key(preprocessed_query):
        completed_queries = find_complete_queries(preprocessed_query, completed_queries)
    
    if len(completed_queries) == 0:
        print("Sorry, nothing to suggest!")
    else:
        [print(f"Count {complete_query[0]} : {complete_query[1]} {complete_query[2]}") for complete_query in completed_queries]
    print('\n')


inp = "trie"
print("Query:", inp)
print("Results:")
complete_user_query(inp, aol_trie)

Query: trie
Results:
Count 5 : tried and true tattoo http://www.triedntruetattoo.com
Count 3 : triest 
Count 3 : triethanalomine http://avalon.unomaha.edu
Count 2 : tried and failed 
Count 2 : when you tried and failed 




## 3. Measure suggest speed ##

Check how fast search is working.

In [9]:
import time

inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]

#TODO: measure avg execution time per query
start = time.time()
for query in inp_queries:
    print("Query:", query)
    print("Results:")
    complete_user_query(query, aol_trie)

avg_time = round((time.time() - start) * 1000 / len(inp_queries), 2)

print(f"Queries are handled in {avg_time} ms on average")

Query: inf
Results:
Count 94 : information clearing house http://www.informationclearinghouse.info
Count 72 : information on training puppy http://www.101-dog-training-tips.com
Count 59 : inflatable slides 
Count 46 : infopass http://infopass.uscis.gov
Count 40 : infolanka http://www.infolanka.com


Query: the best 
Results:
Count 685 : best buy http://www.bestbuy.com
Count 257 : bestcounter.biz 
Count 224 : bestbuy http://www.bestbuy.com
Count 158 : bestbuy.com http://www.bestbuy.com
Count 99 : best western http://www.bestwestern.com


Query: information retrieval
Results:
Sorry, nothing to suggest!


Query: sherlock hol
Results:
Count 4 : sherlock holmes http://www.sherlockian.net
Count 2 : sherlock holmes society http://www.realtime.net
Count 2 : sherlock holmes chronological order http://www.geocities.com
Count 1 : sherlock holmes address 
Count 1 : sherlock holmes audiotapes 


Query: carnegie mell
Results:
Count 6 : carnegie mellon http://www.cmu.edu
Count 1 : carnegie mellon uni

## 4. Adding spellchecking to adviser ##

Let's build Soundex spellchecker, based on the query data provided in the current dataset:

In [10]:
from collections import Counter

def create_vocabulary():
    vocabulary = Counter()
    for query_data in aol_data['processed_query']:
        for term in preprocessor.tokenize(query_data):
            if (len(term)) > 1:
                vocabulary[term] += 1
    print("Vocabulary contains", len(vocabulary), "terms")
    return vocabulary

vocabulary = create_vocabulary()

Vocabulary contains 568101 terms


In [11]:
def edit_dist(s1, s2) -> int:
    """ compute the Damerau-Levenshtein distance between two given strings (s1 and s2)
     Transposition is considered as operation. The snippet is taken from the corresponding Wikipedia page. """ 
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in range(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in range(-1,lenstr2+1):
        d[(-1,j)] = j+1
 
    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )
            if i and j and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + cost) # transposition
 
    return d[lenstr1-1,lenstr2-1]

def produce_soundex_code(word):
    """ Soundex algorithm implementation
     input word, which should be already lowercased
     return Soundex 4-character code, like 'k450' """

    def replace_letters(editable_word: str, 
                        exchange_letters: list, 
                        replace_symbol: str):
        """replace all the exchange_letters in editable_word to 
        replace_symbol."""
        for change_letter in exchange_letters:
            editable_word = editable_word.replace(change_letter, replace_symbol)
        return editable_word
        
    soundex_code = word[0]
    
    if len(word) > 1:
        editable_word = word[1:]
        
        # since "word" can represent some url, it can contains spec. symbols and numbers,
        # let replace it with some additional code.
        spec_symbols = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '-', '@', '\'', '"', '.', ',']
        editable_word = replace_letters(editable_word, spec_symbols, '7')
        
        zero_letters = ['a', 'e', 'i', 'o', 'u', 'h', 'w', 'y']
        editable_word = replace_letters(editable_word, zero_letters, '0')

        one_letters = ['b', 'f', 'p', 'v']
        editable_word = replace_letters(editable_word, one_letters, '1')

        two_letters = ['c', 'g', 'j', 'k', 'q', 's', 'x', 'z']
        editable_word = replace_letters(editable_word, two_letters, '2')

        editable_word = replace_letters(editable_word, ['d', 't'], '3')
        editable_word = replace_letters(editable_word, ['l'], '4')
        editable_word = replace_letters(editable_word, ['m', 'n'], '5')
        editable_word = replace_letters(editable_word, ['r'], '6')

        soundex_code += editable_word[0]
        prev_char_ind = 0
        # Repeatedly save one digit out of each pair of consecutive identical digits.
        for char_ind in range(1, len(editable_word)):
            if editable_word[prev_char_ind] != editable_word[char_ind]:
                soundex_code += editable_word[char_ind]
                prev_char_ind = char_ind
    
    # Pad the resulting string with trailing zeros and return the first four positions
    return (soundex_code.replace('0', '') + '0' * 3)[:4]


def build_soundex_index(dictionary):
    # dictionary is a vocabulary of original words
    # output format: 'code1': ['word1_with_code1', 'word2_with_code1', ...]    

    soundex_index = {}
    for word in dictionary:
        code = produce_soundex_code(word)
        if code not in soundex_index:
            soundex_index[code] = [word]
        else:
            soundex_index[code].append(word)
    return soundex_index


soundex_index = build_soundex_index(vocabulary)


def fix_typo_soundex(word):
    # return words from vocabulary that match with result by soundex fingerprint
    # ordered results by editorial distance
    word_sound_code = produce_soundex_code(word)
    if word_sound_code not in soundex_index:
        return [word]
    
    words_edit_distances = {}  # match similar sound word with its edit. dist. till orig. word
    for similar_sound_word in soundex_index[word_sound_code]:
        words_edit_distances[similar_sound_word] = edit_dist(word, similar_sound_word)
    
    # sort obtained similar words by editorial distance (dict value), and return best match:
    return [k for k, _ in sorted(words_edit_distances.items(), key=lambda item: item[1])][0]

In [12]:
def process_query_with_spellchecker(query):
    preprocessed_query = query.lower()
    fixed_terms = []
    
    for query_term in preprocessor.tokenize(preprocessed_query):
        if query_term in preprocessor.stop_words:
            fixed_terms.append(query_term)  # let's left stop-words as it are.
            continue
        
        fixed_terms.append(fix_typo_soundex(query_term))  # correct query term
    return " ".join(fixed_terms)

### Let's also test spellchecker on some queries: 

In [13]:
inp_queries = ["infa", "cliar hestory", "the best ", "infarmation retrieval", "sherlack holm", "carnegie mell", 
               "babies r", "new york", "googol", "enouhg maney", "USA sta", "Barbara ", "breatany", "soem brikc"]

for query in inp_queries:
    print("Query:", query)
    print("Results:")
    complete_user_query(query, aol_trie)
    
    fixed_query = process_query_with_spellchecker(query)
    print("Query fixed with spellchecker:", fixed_query)
    print("Results with spellchecker:")
    complete_user_query(fixed_query, aol_trie)
    print("------------------------------------------------")

Query: infa
Results:
Count 28 : infatuated 
Count 9 : infantry flash games war http://www.popmatters.com
Count 8 : infant of prague novena http://www.thesacredheart.com
Count 8 : infant dumbo costume http://www.brandsonsale.com
Count 7 : infant easter outfit http://www.bunnycreek.com


Query fixed with spellchecker: info
Results with spellchecker:
Count 94 : information clearing house http://www.informationclearinghouse.info
Count 72 : information on training puppy http://www.101-dog-training-tips.com
Count 46 : infopass http://infopass.uscis.gov
Count 40 : infolanka http://www.infolanka.com
Count 29 : infospace http://www.infospace.com


------------------------------------------------
Query: cliar hestory
Results:
Sorry, nothing to suggest!


Query fixed with spellchecker: clear history
Results with spellchecker:
Count 109 : clear history http://www.hopeforhealing.org
Count 11 : clear my history 
Count 7 : clear history trail http://www.boutell.com
Count 4 : how to clear history 
Cou

So, as you can see the spellchecker works well, and sometimes corrects the queries. For better quality of correction requires more clear data, since current data is fully based on users' queries (which can contain mistakes and noise), sometimes corrections can be wrong (or at least not clear).